## Import Library

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

from xgboost import XGBClassifier

from skimage.feature import graycomatrix, graycoprops, local_binary_pattern

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, learning_curve
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report

## Download Dataset

In [ ]:
import os

TRAIN_PATH = r"C:\Users\HP\Downloads\Brain Tumor DataSet\Training\no_tumor\image(36)"
TEST_PATH  = r"C:\Users\HP\Downloads\Brain Tumor DataSet\Testing"


## Check Size Some Photo

In [3]:
import os
import cv2

TRAIN_PATH = r"C:\Users\HP\Downloads\Brain Tumor DataSet\Training"
folder_name = 'no_tumor'

# Preview first 10 images (recursively)
count = 0
for root, dirs, files in os.walk(os.path.join(TRAIN_PATH, folder_name)):
    for f in files:
        if f.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(root, f)
            img = cv2.imread(img_path)
            if img is not None:
                print(f"{f}: {img.shape}")
                count += 1
            if count == 10:
                break
    if count == 10:
        break


1.jpg: (350, 350, 3)
2.jpg: (350, 350, 3)
3.jpg: (613, 605, 3)
4.jpg: (592, 562, 3)
5.jpg: (442, 442, 3)
6.jpg: (630, 630, 3)
7.jpg: (442, 400, 3)
8.jpg: (212, 220, 3)
image (10).jpg: (225, 225, 3)
image (11).jpg: (198, 254, 3)


## Load Dataset

In [4]:
def load_prep(dataset_path: str):
  label_class = os.listdir(dataset_path)
  class_list = []
  images = []

  kernel_filter = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])

  for i, label in enumerate(label_class):
    label_path = os.path.join(dataset_path, label)
    for f in os.listdir(label_path):
      img_path = os.path.join(label_path, f)
      img = cv2.imread(img_path)
      # Resize
      img = cv2.resize(img, (224, 224))
      # Grayscale
      img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      # Histogram Equalization
      img = cv2.equalizeHist(img)
      # Noise Reduction
      img = cv2.medianBlur(img, 5)
      # Sharpening
      img = cv2.filter2D(img, -1, kernel_filter)
      # Normalization
      img = img / 255.0

      images.append(img)
      class_list.append(i)

  return np.asarray(images), np.asarray(class_list)

In [5]:
train_images, train_labels = load_prep(TRAIN_PATH)
test_images, test_labels = load_prep(TEST_PATH)

## Feature Extraction

In [6]:
def feature_extraction(images: np.ndarray, distances = [1], angles = [0]):
  glcm_features = []
  lbp_features = []

  for img in images:
    img_u8 = (img * 255).astype(np.uint8) if img.max() <= 1 else img.astype(np.uint8)

    # GLCM
    glcm = graycomatrix(img_u8,
                        distances=distances,
                        angles=angles,
                        levels=256,
                        symmetric=True,
                        normed=True)

    contrast = graycoprops(glcm, 'contrast')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    glcm_vector = [contrast, energy, homogeneity, correlation]
    glcm_features.append(glcm_vector)

    # LBP
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(img_u8, n_points, radius, method='uniform')
    hist, _ = np.histogram(
        lbp.ravel(),
        bins=np.arange(0, n_points + 3),
        range=(0, n_points + 2)
    )
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    lbp_features.append(hist)

  glcm_features = np.asarray(glcm_features)
  lbp_features = np.asarray(lbp_features)
  combined = np.hstack((glcm_features, lbp_features))

  return combined

In [7]:
train_features = feature_extraction(train_images)
test_features = feature_extraction(test_images)

## Scaling

In [8]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

## Modeling

In [9]:
model_svm = SVC(random_state=42)
model_random_forest = RandomForestClassifier(n_estimators=200, random_state=42)
model_xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    random_state=42,
    eval_metric='logloss'
)

model_svm.fit(train_features, train_labels)
model_random_forest.fit(train_features, train_labels)
model_xgb.fit(train_features, train_labels)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [10]:
svm_pred = model_svm.predict(test_features)
svm_acc = accuracy_score(test_labels, svm_pred)
svm_recall = recall_score(test_labels, svm_pred, average='macro')
svm_precision = precision_score(test_labels, svm_pred, average='macro')
svm_f1 = f1_score(test_labels, svm_pred, average='macro')
print(f'SVM Accuracy Score: {svm_acc}')
print()
print(f'SVM Recall Score: {svm_recall}')
print()
print(f'SVM Precision Score: {svm_precision}')
print()
print(f'SVM F1 Score: {svm_f1}')

SVM Accuracy Score: 0.4365482233502538

SVM Recall Score: 0.4477925130099043

SVM Precision Score: 0.4832247899159664

SVM F1 Score: 0.4204168683661713


In [11]:
print('SVM Classification Report')
print(classification_report(test_labels, svm_pred))

SVM Classification Report
              precision    recall  f1-score   support

           0       0.57      0.20      0.30       100
           1       0.43      0.25      0.32       115
           2       0.38      0.77      0.50       105
           3       0.56      0.57      0.56        74

    accuracy                           0.44       394
   macro avg       0.48      0.45      0.42       394
weighted avg       0.47      0.44      0.41       394



## Hyperparameter Tuning

### SVM

In [12]:
svc = SVC()

param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = GridSearchCV(
    svc,
    param_grid,
    cv=cv,
    scoring='accuracy'
)

search.fit(train_features, train_labels)
print(search.best_params_)
print(search.best_score_)

{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
0.8153310104529616


In [13]:
svc_tuned = SVC(kernel='rbf', C=10, gamma=1)
svc_tuned.fit(train_features, train_labels)

svc_tuned_acc = accuracy_score(test_labels, svc_tuned.predict(test_features))
svc_tuned_recall = recall_score(test_labels, svc_tuned.predict(test_features), average='macro')
svc_tuned_precision = precision_score(test_labels, svc_tuned.predict(test_features), average='macro')
svc_tuned_f1 = f1_score(test_labels, svc_tuned.predict(test_features), average='macro')

print(f'Accuracy SVM Tuned: {svc_tuned_acc}')
print(f'Recall SVM Tuned: {svc_tuned_recall}')
print(f'Precision SVM Tuned: {svc_tuned_precision}')
print(f'F1 SVM Tuned: {svc_tuned_f1}')

Accuracy SVM Tuned: 0.7106598984771574
Recall SVM Tuned: 0.6928070617201052
Precision SVM Tuned: 0.7794214061133746
F1 SVM Tuned: 0.6620855484844466


In [14]:
import numpy as np

# Example feature matrix (100 samples, 20 features)
X = np.random.rand(100, 20)
y = np.random.randint(0, 2, 100)


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

model = SVC(kernel="rbf", probability=True)
model.fit(X_train_scaled, y_train)


,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,True
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [17]:
import os
import sys

PROJECT_DIR = r"C:\Users\HP\Downloads\brain-tumor1\BrainTumorClassifierAI-main"
os.chdir(PROJECT_DIR)
 
# Add project root to Python path
sys.path.insert(0, PROJECT_DIR)
 
print("Current dir:", os.getcwd())
 
from utils.preprocessing import preprocess_image
from utils.features import feature_extraction
 
print("IMPORT SUCCESS")

Current dir: C:\Users\HP\Downloads\brain-tumor1\BrainTumorClassifierAI-main
IMPORT SUCCESS


In [20]:
import os
import sys
import cv2
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# -----------------------------
# PROJECT PATHS
# -----------------------------
PROJECT_DIR = r"C:\Users\HP\Downloads\brain-tumor1\BrainTumorClassifierAI-main"
TRAIN_PATH = r"C:\Users\HP\Downloads\Brain Tumor DataSet\Training"
MODEL_DIR = os.path.join(PROJECT_DIR, "models")
os.makedirs(MODEL_DIR, exist_ok=True)

# Add project directory to Python path so utils can be imported
sys.path.append(PROJECT_DIR)
from utils.preprocessing import preprocess_image
from utils.features import feature_extraction

# -----------------------------
# LABEL MAPPING (match folder names)
# -----------------------------
LABEL_MAP = {
    "pituitary_tumor": 0,
    "no_tumor": 1,
    "meningioma_tumor": 2,
    "glioma_tumor": 3
}

# -----------------------------
# LOAD TRAINING IMAGES & EXTRACT FEATURES
# -----------------------------
X_train, y_train = [], []

for class_name, label in LABEL_MAP.items():
    folder = os.path.join(TRAIN_PATH, class_name)
    if not os.path.exists(folder):
        print(f"⚠️ Folder not found: {folder}")
        continue

    # Recursively walk through all subfolders and files
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                if img is None:
                    continue

                # Preprocess & extract features
                img_resized = preprocess_image(img, target_size=(224, 224))
                features = feature_extraction(img_resized)
                X_train.append(features[0])  # flatten array
                y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

print("✅ Training feature shape:", X_train.shape)
print("✅ Training labels shape:", y_train.shape)

# -----------------------------
# SCALE + TRAIN SVM
# -----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

model = SVC(kernel="rbf", probability=True)
model.fit(X_train_scaled, y_train)
print("✅ Model trained successfully")

# -----------------------------
# SAVE MODEL + SCALER
# -----------------------------
MODEL_PATH = os.path.join(MODEL_DIR, "model_bundle.pkl")
joblib.dump(
    {"model": model, "scaler": scaler},
    MODEL_PATH
)

print(f"✅ model_bundle.pkl saved successfully under: {MODEL_PATH}")


✅ Training feature shape: (2870, 14)
✅ Training labels shape: (2870,)
✅ Model trained successfully
✅ model_bundle.pkl saved successfully under: C:\Users\HP\Downloads\brain-tumor1\BrainTumorClassifierAI-main\models\model_bundle.pkl
